In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

In [ ]:
from multiversx_sdk_core import Address, Transaction, TransactionPayload
from multiversx_sdk_core.transaction_builders import DefaultTransactionBuildersConfiguration
from multiversx_sdk_core import TokenPayment
from multiversx_sdk_network_providers import ProxyNetworkProvider, ApiNetworkProvider
from tools.notebooks import env
import config

builder_config = DefaultTransactionBuildersConfiguration(chain_id="1")
proxy_mainnet = ProxyNetworkProvider("https://gateway.multiversx.com")
api_mainnet = ApiNetworkProvider("https://api.multiversx.com")
proxy_shadowfork = ProxyNetworkProvider(config.DEFAULT_PROXY)

In [ ]:
from multiversx_sdk_core import AccountNonceHolder
from multiversx_sdk_wallet import UserSigner
from pathlib import Path

user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))

In [10]:
def prep_forward_tx(tx_hash: str) -> Transaction:
    original_tx_data = proxy_mainnet.get_transaction(tx_hash)
    print(original_tx_data.data)
    forwarded_tx  = Transaction(
        sender=original_tx_data.sender,
        receiver=original_tx_data.receiver,
        value=TokenPayment.egld_from_integer(original_tx_data.value),
        data=TransactionPayload.from_str(original_tx_data.data),
        chain_id=builder_config.chain_id,
        gas_limit=original_tx_data.gas_limit,
        gas_price=builder_config.min_gas_price
    )

    print(forwarded_tx.to_dictionary())
    return forwarded_tx


def sign_and_send(tx: Transaction, account: UserSigner):
    user_nonce_holder = AccountNonceHolder(proxy_shadowfork.get_account(tx.sender).nonce)
    tx.nonce = user_nonce_holder.get_nonce_then_increment()
    tx.signature = account.sign(tx)
    proxy_shadowfork.send_transaction(tx)

clone & forward transaction

In [ ]:
hash = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

forwarded_tx = prep_forward_tx(hash)

In [ ]:
sign_and_send(forwarded_tx, user_signer)

clone & forward transactions on account

In [11]:
from time import sleep
contract_address = "erd1qqqqqqqqqqqqqpgqapxdp9gjxtg60mjwhle3n6h88zch9e7kkp2s8aqhkg"

transactions = api_mainnet.get_transactions(Address.from_bech32(contract_address))
for transaction in transactions:
    print(f"Forwarded tx hash: {transaction.hash}")
    forwarded_tx = prep_forward_tx(transaction.hash)
    sign_and_send(forwarded_tx, user_signer)
    sleep(1)

Forwarded tx hash: d6dfa9b82ca6191ee7ed693c30ba9b327846ee0d6afdc6ad3e8dfeb5d089cb6f
ESDTNFTTransfer@45474c444d4558464c2d633235323165@04b3c8@482728580e8145f2@00000000000000000500e84cd0951232d1a7ee4ebff319eae738b172e7d6b055@636c61696d52657761726473
OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1dzd6r8j7fvm26wa7vgs5j33kmvh8p8hxtupe3u8v7kvlyqx2yazqckyz39'), ('sender', 'erd1dzd6r8j7fvm26wa7vgs5j33kmvh8p8hxtupe3u8v7kvlyqx2yazqckyz39'), ('gasPrice', 1000000000), ('gasLimit', 22000000), ('data', 'RVNEVE5GVFRyYW5zZmVyQDQ1NDc0YzQ0NGQ0NTU4NDY0YzJkNjMzMjM1MzIzMTY1QDA0YjNjOEA0ODI3Mjg1ODBlODE0NWYyQDAwMDAwMDAwMDAwMDAwMDAwNTAwZTg0Y2QwOTUxMjMyZDFhN2VlNGViZmYzMTllYWU3MzhiMTcyZTdkNmIwNTVANjM2YzYxNjk2ZDUyNjU3NzYxNzI2NDcz'), ('chainID', '1'), ('version', 1), ('signature', '')])
Forwarded tx hash: b056c70ff1613f8fcc103c84fdc5135634e5cd2bd614a59294b1dfa50c3ec387
ESDTNFTTransfer@45474c444d4558464c2d633235323165@0382c1@2f99bde26bce8b45@00000000000000000500e84cd0951232d1a7ee4ebff319eae738b172e7d6b

KeyboardInterrupt: 

ESDT Transfer

In [6]:
from multiversx_sdk_core.transaction_builders import MultiESDTNFTTransferBuilder, DefaultTransactionBuildersConfiguration
from multiversx_sdk_core import TokenPayment, Address
from multiversx_sdk_network_providers import ProxyNetworkProvider
from multiversx_sdk_core import AccountNonceHolder
from multiversx_sdk_wallet import UserSigner
from pathlib import Path

config = DefaultTransactionBuildersConfiguration(chain_id='1')
proxy = ProxyNetworkProvider('https://proxy-shadowfork-one.elrond.ro')
user_signer = UserSigner.from_pem_file(Path("~/Documents/sh1.pem"))
sender = Address.from_bech32('erd1ezp86jwmcp4fmmu2mfqz0438py392z5wp6kzuqsjldgd68nwt89qshfs0y')
receiver = Address.from_bech32('erd1hfnw32gkydnj80cljjvkfz9kl3tachmwcpgjhxm5k5l0vhu4pr9s4zwxwa')

transfers = [
    TokenPayment.meta_esdt_from_amount('RIDE-7d18e9', 0, 1000, 18)
             ]


user_nonce_holder = AccountNonceHolder(proxy.get_account(sender).nonce)

builder = MultiESDTNFTTransferBuilder(
    config,
    destination=receiver,
    sender=sender,
    gas_limit=30000000,
    payments=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1ezp86jwmcp4fmmu2mfqz0438py392z5wp6kzuqsjldgd68nwt89qshfs0y'), ('sender', 'erd1ezp86jwmcp4fmmu2mfqz0438py392z5wp6kzuqsjldgd68nwt89qshfs0y'), ('gasPrice', 1000000000), ('gasLimit', 30000000), ('data', 'TXVsdGlFU0RUTkZUVHJhbnNmZXJAYmE2NmU4YTkxNjIzNjcyM2JmMWY5NDk5NjQ4OGI2ZmM1N2RjNWY2ZWMwNTEyYjliNzRiNTNlZjY1Zjk1MDhjYkAwMUA1MjQ5NDQ0NTJkMzc2NDMxMzg2NTM5QEAzNjM1YzlhZGM1ZGVhMDAwMDA='), ('chainID', '1'), ('version', 2), ('signature', '')])
Transaction data: MultiESDTNFTTransfer@ba66e8a916236723bf1f94996488b6fc57dc5f6ec0512b9b74b53ef65f9508cb@01@524944452d376431386539@@3635c9adc5dea00000


'38bbf7952ef80842804de2bdcd5d65077f96da89e7599079b1184bf743ecfaf6'

EGLD Transfer

In [8]:
from multiversx_sdk_core.transaction_builders import EGLDTransferBuilder, DefaultTransactionBuildersConfiguration
from multiversx_sdk_core import TokenPayment, Address
from multiversx_sdk_network_providers import ProxyNetworkProvider
from multiversx_sdk_core import AccountNonceHolder
from multiversx_sdk_wallet import UserSigner
from pathlib import Path

config = DefaultTransactionBuildersConfiguration(chain_id='1')
proxy = ProxyNetworkProvider('https://proxy-shadowfork-one.elrond.ro')
user_signer = UserSigner.from_pem_file(Path("~/Documents/sh1.pem"))
sender = Address.from_bech32('erd1ezp86jwmcp4fmmu2mfqz0438py392z5wp6kzuqsjldgd68nwt89qshfs0y')
receiver = Address.from_bech32('erd1hfnw32gkydnj80cljjvkfz9kl3tachmwcpgjhxm5k5l0vhu4pr9s4zwxwa')

transfer = TokenPayment.egld_from_amount(10)


user_nonce_holder = AccountNonceHolder(proxy.get_account(sender).nonce)

builder = EGLDTransferBuilder(
    config,
    receiver=receiver,
    sender=sender,
    gas_limit=30000000,
    payment=transfer
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

Transaction: OrderedDict([('nonce', 0), ('value', '10000000000000000000'), ('receiver', 'erd1hfnw32gkydnj80cljjvkfz9kl3tachmwcpgjhxm5k5l0vhu4pr9s4zwxwa'), ('sender', 'erd1ezp86jwmcp4fmmu2mfqz0438py392z5wp6kzuqsjldgd68nwt89qshfs0y'), ('gasPrice', 1000000000), ('gasLimit', 30000000), ('chainID', '1'), ('version', 2), ('signature', '')])
Transaction data: 


'090451fba0a61e4bf5bdf82dd6d710e98d829aabc6901b8606a2218b073c7d2d'